In [4]:
import sys
import os

# Añade el path absoluto a la carpeta donde está el archivo
sys.path.append(os.path.abspath("../MTP/Multi_Task_Pretrain/backbone"))

# Importa correctamente el modelo
from vit_win_rvsa_v3_wsz7 import vit_b_rvsa

In [4]:
import torch
import torch.nn as nn
import sys
import os

# === Agregar la ruta absoluta a la carpeta 'backbone' ===
sys.path.append(os.path.abspath("../MTP/Multi_Task_Pretrain/backbone"))

from vit_win_rvsa_v3_wsz7 import vit_b_rvsa

# === Paso 1: preparar argumentos para el backbone ===
class Args:
    image_size = 224
    use_ckpt = "False"

args = Args()

# === Paso 2: crear el backbone ===
backbone = vit_b_rvsa(args, inchannels=3)

# === Paso 3: definir modelo completo (backbone + clasificador) ===
class ModeloClasificacion(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.classifier = nn.Linear(backbone.out_channels[0], num_classes)

    def forward(self, x):
        features = self.backbone(x)[0]
        pooled = nn.functional.adaptive_avg_pool2d(features, 1).view(x.size(0), -1)
        return self.classifier(pooled)

# === Paso 4: crear el modelo y enviarlo al dispositivo ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModeloClasificacion(backbone, num_classes=10).to(device)


C:\Users\diana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\diana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used


In [7]:
from torchvision.datasets import EuroSAT

from torchvision import transforms
import os

root_path = "../data"

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Tamaño esperado por ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalización de ImageNet
])

# Descargar el dataset
dataset = EuroSAT(root=root_path, download=True, transform=transform)

In [8]:
from torch.utils.data import random_split

# 80% train, 20% val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [9]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
import time
import torch.nn as nn
import torch.optim as optim
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10

# === Early Stopping ===
best_val_acc = 0.0
patience = 3
epochs_without_improvement = 0
best_model_path = "mejor_modelo.pth"

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if (i+1) % 10 == 0 or (i+1) == len(train_loader):
            print(f"[{epoch+1}/{num_epochs}] Step {i+1}/{len(train_loader)} - Loss: {loss.item():.4f}", flush=True)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    duration = time.time() - start_time
    print(f"Época {epoch+1} finalizada | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}% | Tiempo: {duration:.1f}s", flush=True)

    # === Validación ===
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_inputs, val_targets in val_loader:
            val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
            val_outputs = model(val_inputs)
            _, val_predicted = val_outputs.max(1)
            val_total += val_targets.size(0)
            val_correct += val_predicted.eq(val_targets).sum().item()

    val_acc = 100. * val_correct / val_total
    print(f"Validación - Acc: {val_acc:.2f}%\n{'-'*60}", flush=True)

    # === Early stopping ===
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_without_improvement = 0
        torch.save(model.state_dict(), best_model_path)
        print(f"Nuevo mejor modelo guardado con Acc: {best_val_acc:.2f}%", flush=True)
    else:
        epochs_without_improvement += 1
        print(f"Sin mejora en validación ({epochs_without_improvement}/{patience})", flush=True)
        if epochs_without_improvement >= patience:
            print("Early stopping activado.")
            break


[1/20] Step 10/675 - Loss: 2.0683
[1/20] Step 20/675 - Loss: 1.7672
